In [5]:
!pip install dash
!pip install "dash-bootstrap-components<1"

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_i

In [2]:
# Loading libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import numpy as np
import scipy.stats as ss

/usr/local/lib/python3.10/dist-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [3]:
# Creating a Class OP_pricing
class OP_pricing:
    def __init__(self, S0, r, sigma, T, K=None, Ite=10000, t=0):
        self.S0 = S0
        self.r = r
        self.sigma = sigma
        self.T = T
        self.K = K
        self.Ite = Ite
        self.t = t

    def bs_mc(self, opttype):
        np.random.seed(42)

        days = int((self.T - self.t) * 252)
        dt = 1 / 252
        ST = np.zeros(self.Ite)

        for i in range(self.Ite):
            St = self.S0
            for _ in range(days):
                z = np.random.normal(0, 1)
                St = St * np.exp((self.r - 0.5 * self.sigma**2) * dt + self.sigma * np.sqrt(dt) * z)
            ST[i] = St

        d1 = (np.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma**2) * (self.T - self.t)) / (self.sigma * np.sqrt(self.T - self.t))

        if opttype == "C":
            payoffs = np.maximum(ST - self.K, 0)
            Delta = ss.norm.cdf(d1)
            Vega = self.S0 * ss.norm.pdf(d1) * np.sqrt(self.T)
        else:
            payoffs = np.maximum(self.K - ST, 0)
            Delta = -ss.norm.cdf(-d1)
            Vega = self.S0 * ss.norm.pdf(d1) * np.sqrt(self.T)

        average_payoff = np.mean(payoffs)
        price = np.exp(-self.r * (self.T - self.t)) * average_payoff

        return price, Delta, Vega

    def Heston_paths(self, v0, kappa, theta, sigma_v, rho):
        M0 = 252
        M = int(M0 * self.T)
        dt = self.T / M
        rand = np.random.standard_normal((2, M + 1, self.Ite))

        covariance_matrix = np.array([[1.0, rho], [rho, 1.0]])
        cho_matrix = np.linalg.cholesky(covariance_matrix)

        def SDE_vol(v0, kappa, theta, sigma_v):
            v = np.zeros((M + 1, self.Ite), dtype=np.float64)
            v[0] = v0
            sdt = np.sqrt(dt)
            for t in range(1, M + 1):
                ran = np.dot(cho_matrix, rand[:, t])
                v[t] = np.maximum(0, v[t - 1] + kappa * (theta - v[t - 1]) * dt + np.sqrt(v[t - 1]) * sigma_v * ran[1] * sdt)
            return v

        def paths(S0, v):
            S = np.zeros((M + 1, self.Ite), dtype=np.float64)
            S[0] = S0
            sdt = np.sqrt(dt)
            for t in range(1, M + 1):
                ran = np.dot(cho_matrix, rand[:, t])
                S[t] = S[t - 1] * np.exp((self.r - 0.5 * v[t - 1]) * dt + np.sqrt(v[t - 1]) * ran[0] * sdt)
            return S

        V = SDE_vol(v0, kappa, theta, sigma_v)
        S = paths(self.S0, V)

        return S

    def heston_mc(self, S, opttype):
        if opttype == "C":
            payoff = np.maximum(0, S[-1, :] - self.K)
            average = np.mean(payoff)
        else:
            payoff = np.maximum(self.K - S[-1, :], 0)
            average = np.mean(payoff)

        return np.exp(-self.r * (self.T - self.t)) * average


    def merton_paths(self, lamb, mu, delta):
        M = 252
        dt = self.T / M
        SM = np.zeros((M + 1, self.Ite))
        SM[0] = self.S0

        rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)
        z1 = np.random.standard_normal((M + 1, self.Ite))
        z2 = np.random.standard_normal((M + 1, self.Ite))
        y = np.random.poisson(lamb * dt, (M + 1, self.Ite))

        for t in range(1, M + 1):
            SM[t] = SM[t - 1] * (
                np.exp((self.r - rj - 0.5 * self.sigma**2) * dt + self.sigma * np.sqrt(dt) * z1[t])
                + (np.exp(mu + delta * z2[t]) - 1) * y[t]
            )
            SM[t] = np.maximum(SM[t], 0.00001)

        return SM

    def merton_mc(self, S, opttype):
        if opttype == "C":
            payoff = np.maximum(0, S[-1, :] - self.K)
            average = np.mean(payoff)
        else:
            payoff = np.maximum(self.K - S[-1, :], 0)
            average = np.mean(payoff)

        return np.exp(-self.r * (self.T - self.t)) * average

In [4]:
# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H1("Option Pricing App"), className="mb-4")),
    dbc.Row([
        dbc.Col([
            dbc.Form([
                dbc.Label("Model"),
                dcc.Dropdown(id="model", options=[
                    {'label': 'Black-Scholes', 'value': 'bs'},
                    {'label': 'Heston', 'value': 'heston'},
                    {'label': 'Merton', 'value': 'merton'}
                ], value='bs', className="form-control mb-3"),

                dbc.Label("Initial stock price (S0)"),
                dbc.Input(id="S0", type="number", value=100.0, step=0.01, className="form-control mb-3"),

                dbc.Label("Risk-free rate (r)"),
                dbc.Input(id="r", type="number", value=0.05, step=0.0001, className="form-control mb-3"),

                dbc.Label("Volatility (sigma)"),
                dbc.Input(id="sigma", type="number", value=0.2, step=0.0001, className="form-control mb-3"),

                dbc.Label("Time to maturity (T)"),
                dbc.Input(id="T", type="number", value=1.0, step=0.01, className="form-control mb-3"),

                dbc.Label("Strike price (K)"),
                dbc.Input(id="K", type="number", value=100.0, step=0.01, className="form-control mb-3"),

                dbc.Label("Lambda (lamb)"),
                dbc.Input(id="lamb", type="number", value=0.2, step=0.0001, className="form-control mb-3"),

                dbc.Label("Mu (mu)"),
                dbc.Input(id="mu", type="number", value=0.1, step=0.0001, className="form-control mb-3"),

                dbc.Label("Option type"),
                dcc.Dropdown(id="opttype", options=[
                    {'label': 'Call', 'value': 'C'},
                    {'label': 'Put', 'value': 'P'}
                ], value='C', className="form-control mb-3"),

                dbc.Button("Price Option", id="price-btn", color="primary", className="mt-3")
            ])
        ], width=4),
        dbc.Col([
            html.H2("Option Pricing Result"),
            html.Div(id="price-output")
        ])
    ])
])

@app.callback(
    Output("price-output", "children"),
    Input("price-btn", "n_clicks"),
    State("model", "value"),
    State("S0", "value"),
    State("r", "value"),
    State("sigma", "value"),
    State("T", "value"),
    State("K", "value"),
    State("lamb", "value"),
    State("mu", "value"),
    State("opttype", "value")
)
def update_price(n_clicks, model, S0, r, sigma, T, K, lamb, mu, opttype):
    if n_clicks is None:
        return ""

    Ite = 10000
    t = 0
    op_pricing = OP_pricing(S0, r, sigma, T, K, Ite, t)

    if model == 'bs':
        price, Delta, Vega = op_pricing.bs_mc(opttype)
    elif model == 'heston':
        v0, kappa, theta, sigma_v, rho = 0.2, 2, 0.2, 0.1, -0.5  # Sample Heston parameters
        S = op_pricing.Heston_paths(v0, kappa, theta, sigma_v, rho)
        price = op_pricing.heston_mc(S, opttype)
    elif model == 'merton':
        S = op_pricing.merton_paths(lamb, mu)
        price = op_pricing.merton_mc(S, opttype)

    return f"The price of a {opttype} option for the {model} model is {price:.3f}"

if __name__ == "__main__":
    app.run_server(debug=True)

<IPython.core.display.Javascript object>